In [1]:
import pandas as pd
import sql_functions as sql_int
import Levenshtein

## Get Names from Tournament


take the playerlist and reduce duplikates in it

In [2]:
player_import = pd.read_csv('/program/tft_overview/tft_Tables/data_sheets/name_list_GSC2.csv',
                         dtype={'liquipedia_name' : str,'country_flag' : str} )
df_player_import = player_import.copy()

In [3]:
df_player_import

,import_name
0,5454
1,valentin3411
2,Memo
3,Kingdanzz
4,Aexoq
...,...
123,Epik
124,keenko
125,Littletoof
126,DeliryummEUW


In [4]:
df_player_import = df_player_import.rename(columns={'liquipedia_name' : 'import_name'})
#df_player_import['import_name_lower'] = df_player_import['import_name'].apply(lambda x: x.lower() if isinstance(x, str) else x)
df_player_import = df_player_import.drop_duplicates()

Same liquipedia name but different country correct the entrys

In [5]:
df_player_import[df_player_import.duplicated(keep = False, subset=['import_name']) == True].sort_values(by='import_name')

,import_name


#### delete double entrys

In [6]:
df_player_import.drop([148],inplace=True)


## Compare Names with *alternative_names*

In [6]:
schema = 'tft_table'
sql_query = f'SELECT * FROM {schema}.lobby;'

alternative_name = pd.DataFrame(sql_int.get_data_alchemy(sql_query),
                                    columns=['player_name','liquipedia_name','names_id','player_id'])
df_alternative_name = alternative_name.copy()

#df_alternative_name[['player_name','liquipedia_name']] = df_alternative_name[['player_name','liquipedia_name']].apply(lambda x: x.str.lower())


replace the imported names with liquipedia_names from the alternative list.   
If there was someone with his alternative name in, they have now liquipedia_name

In [7]:
df_merge_import_alternative = (pd.merge(df_player_import['import_name'], df_alternative_name[['player_name','liquipedia_name']],
                                         how="left", left_on='import_name',right_on='player_name')
                               )

df_merge_import_alternative.loc[df_merge_import_alternative['player_name'].notna(), 'import_name'] = df_merge_import_alternative['liquipedia_name']
df_merge_import_alternative = df_merge_import_alternative.drop(columns=['player_name','liquipedia_name']).drop_duplicates()
df_merge_import_alternative

,import_name
0,5454
1,valentin3411
2,Memo
3,Kingdanzz
4,Aexoq
...,...
123,Epik
124,keenko
125,Littletoof
126,DeliryummEUW


2nd check cause 2nd merge

In [8]:
df_merge_import_alternative[df_merge_import_alternative.duplicated(keep= False, subset=['import_name']) == True].sort_values(by='import_name')

,import_name


## Search Names in *players*

In [9]:
sql_query = f'''select liquipedia_name, player_id from {schema}.player;'''


liquipedia_name = pd.DataFrame(sql_int.get_data_alchemy(sql_query),
                                columns=['liquipedia_name','player_id'])

df_liquipedia_name = liquipedia_name.copy()
##df_liquipedia_name['liquipedia_name'] = df_liquipedia_name['liquipedia_name'].apply(lambda x: x.lower() if isinstance(x, str) else x)

df_merge_import_alternative_liquipedia =pd.merge(
                                            df_merge_import_alternative,df_liquipedia_name['liquipedia_name'], how='left',
                                            left_on= 'import_name', right_on='liquipedia_name')

df_new_players are all players we put in over our names_list, which arn't in *players* 

In [10]:
df_new_player = df_merge_import_alternative_liquipedia[df_merge_import_alternative_liquipedia['liquipedia_name'] 
                                                         .isna()].drop(columns=['liquipedia_name'])

## Push new Names to *alternative_names*

In [11]:
list_new_player = df_new_player['import_name'].to_list()
list_liquipedia_name = df_liquipedia_name['liquipedia_name'].to_list()
list_new_player

 Check similarity 

In [13]:
similar_name = []
for i in list_new_player :
    for j in list_liquipedia_name:
        if i == j:
            pass
        elif  Levenshtein.jaro(i,j) > 0.75:
            similar_name.append([j,i])
df_similar_name = pd.DataFrame(similar_name,columns=['liquipedia_name','import_name'])

#### Controll
***This list must be controlled before pushed!***

In [14]:
df_similar_name

,liquipedia_name,import_name
0,L3S Coco,L3Scoco
1,l3s coco,L3Scoco
2,Texita,texita2
3,Vomendeth,Vomendethh
4,simplyw0jtek,simplywojtek
5,LastKardax,Laras
6,Betrayal,betrayal_tft
7,Xus0o,xus0o
8,Double61,KCDouble61
9,Ryko,Zyko


Do some magic here

In [15]:
df_similar_name = df_similar_name.iloc[[0,1]]

df_similar_name

,liquipedia_name,import_name
0,TSAROU,TSAROU WINS
1,ZeulEnache,BG ZeulEnache


#### Data push
**Hier ist der Push in die Database, be sure**

The datafram has to be established

In [16]:
## ACHTUNG é ist das selbe wie e
df_similar_name_push = pd.merge(
                                df_similar_name,df_liquipedia_name[['liquipedia_name','player_id']], how='left',
                                left_on= 'liquipedia_name', right_on='liquipedia_name').drop(columns=['liquipedia_name'])



In [17]:
#df_player_import['name_lower'] = df_player_import['import_name'].apply(lambda x : x.lower())

df_similar_name_push = (pd.merge(df_similar_name_push,df_player_import,how='left', left_on='import_name',right_on='import_name')
                        .rename(columns={'import_name': 'player_name'}))


In [18]:
df_similar_name_push

,player_name,player_id
0,TSAROU WINS,90
1,BG ZeulEnache,141
2,LiquidSnoodyboo,105
3,IZI Lyyyress,49
4,S1C ODESZA,79
5,ANO shreddin,117
6,Ging GG1,37
7,SLY Xperion,85
8,OPS Jimray3,51
9,br0ken99,18


In [ ]:
                         .drop(columns=['import_name_x','name'])
                         .rename(columns={'import_name': 'player_name'})

In [21]:
sql_int.push_to_database(df_similar_name_push,'alternative_name',sql_int.get_engine_alchemy(),schema)


The alternative_name table was imported successfully.


## Push new names in *player*

In [21]:
df_new_player = pd.merge(df_new_player,df_similar_name, how='left',
                              left_on= 'import_name', right_on='import_name')  ## on player name cause all with same liquipedia_name are already out
df_new_player = df_new_player[df_new_player['liquipedia_name'].isna()]
df_new_player = df_new_player.drop(columns=['liquipedia_name'])

df_new_player = pd.merge(df_new_player,df_player_import,how='left',)
new_player_push = df_new_player.rename(columns={'import_name' : 'liquipedia_name','country_flag' : 'country_id'})

In [22]:
new_player_push

,liquipedia_name
0,z10 lelouch
1,l3s coco
2,S1C xus0o
3,SNG HYPNO
4,IZI Bränk
5,Aug1


In [23]:
sql_int.push_to_database(new_player_push,'player',sql_int.get_engine_alchemy(),schema)

The player table was imported successfully.


## Push these new names in alternative_names

In [24]:
sql_query = f'''select liquipedia_name, player_id from {schema}.player;'''
df_liquipedia_name = pd.DataFrame(sql_int.get_data_alchemy(sql_query),
                                columns=['liquipedia_name','player_id'])
df_alternative_name_push = (pd.merge(new_player_push['liquipedia_name'],df_liquipedia_name,how='left', left_on='liquipedia_name',right_on='liquipedia_name')
                             .rename(columns={'liquipedia_name':'player_name'})
                             )

In [25]:
sql_int.push_to_database(df_alternative_name_push,'alternative_name',sql_int.get_engine_alchemy(),schema)

The alternative_names table was imported successfully.


In [ ]:
df #updating data in dataframe wäre der dataframe mit den liquid_namen mit flagge. (die vorher keine gehabt haben)

s_update = "" #String of updations

# Loop through the data frame

for i in range(len(df)):
    s_update += "update players set country_name = '%s' where liquipedia_name = '%s';" %(df[country_name][i], df[liquipedia_name][i])